In [32]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [33]:
import pandas as pd
import torch

data_path_train = "/kaggle/input/digit-recognizer/train.csv"
train_data = pd.read_csv(data_path_train)
train_data.head(10)



,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
data_path_test = "/kaggle/input/digit-recognizer/test.csv"
test_data = pd.read_csv(data_path_test)
test_data.head(10)


,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
train_data.shape

(42000, 785)

In [36]:
test_data.shape

(28000, 784)

In [37]:
from torch.utils.data import DataLoader

# Ensure correct number of columns
X_train = train_data.iloc[:, 1:].values  # All rows, columns from 1 to 784 (features)
y_train = train_data.iloc[:, 0].values  # All rows, column 0 (labels)

X_test = test_data.iloc[:, :784].values  # Ensure only the first 784 columns
y_test = test_data.iloc[:, 0].values  # Labels (if available)

# Convert to PyTorch tensors and reshape
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).view(-1, 1, 28, 28)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32).view(-1, 1, 28, 28)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Normalize pixel values
X_train_tensor /= 255.0
X_test_tensor /= 255.0

# Create TensorDataset
train_data = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
test_data = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)



In [38]:
from torch.utils.data import DataLoader

loaders = {
    'train': DataLoader(
        train_data,
        batch_size=100,
        shuffle=True,
        num_workers=0,
    ),

    'test':DataLoader(
        test_data,
        batch_size=100,
        shuffle=False,
        num_workers=0,
            
        
    )
}

In [39]:
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7d1be4ebad10>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7d1be4ebb7c0>}

In [40]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

(a) Input: raw pixel values are provided as input. 
(b) Convolutional layer: Input layers translates the results of neuron layer. There is a need to specify the filter to be used. Each filter can only be a 5*5 window that will slide over input data and get pixels with maximum intensities. 
(c) Rectified linear unit [ReLU] layer: provided activation function on the data taken as an image. In the case of backpropagation, ReLU function is used which prevents the values of pixels form changing. 
(d) Pooling layer: Performs a down-sampling operation in volume along the dimensions (width, height). 
(e) Fully connected layer: score class is focused, and a maximum score of the input digits is found

Found in a paper named : Handwritten digit classification using Convolutional Neural Networks

In [41]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
#         # First convolutional layer: 
#         # Takes input with 1 channel (grayscale image) and outputs 10 feature maps, 
#         # using a 5x5 kernel to process the input
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)  # Input: 1x28x28, Output: 10x24x24 using formula (input_size - 2* (kernel) / padding )+ 1
#         # Second convolutional layer: 
#         # Takes 10 feature maps from the previous layer and outputs 20 feature maps
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)  # Input: 10x24x24, Output: 20x20x20

        
#         # Dropout layer: Randomly zeroes some of the features during training 
#         # to prevent overfitting and improve generalization
        self.conv2_drop = nn.Dropout2d()
        
#         # Fully connected layer 1: 
#         # Flattens the output from the convolutional layers to 320 features, 
#         # then reduces it to 50 features
#         # the reason to 320 is : 
        self.fc1 = nn.Linear(20 * 4 * 4, 50) 
#         # Fully connected layer 2:
#         # Reduces the 50 features down to 10 (for classification into 10 classes)
        self.fc2 = nn.Linear(50, 10)  # Output layer with 10 units for 10 classes (digits 0-9)
#   

    
    def forward(self, x):
        x = F.relu(self.conv1(x))  # Output: 10x24x24
        x = F.max_pool2d(x, 2)     # Max pooling: 10x12x12
        x = F.relu(self.conv2(x))  # Output: 20x8x8
        x = F.max_pool2d(x, 2)     # Max pooling: 20x4x4
        x = self.conv2_drop(x)     # Apply dropout
        x = x.view(x.size(0), -1)  # Flatten the tensor for fully connected layers
        x = F.relu(self.fc1(x))    # Fully connected layer
        x = self.fc2(x)            # Output layer (no softmax because it's handled by CrossEntropyLoss)
        return x


In [42]:
import torch 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.005)
loss_fn= nn.CrossEntropyLoss()





In [43]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loaders['train']):
        # Reshape input
        data = data.view(-1, 1, 28, 28)  # Reshape to (batch_size, 1, 28, 28)
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()  # Backpropagation
        optimizer.step()  # Update parameters
        
        if batch_idx % 20 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(loaders['train'].dataset)} "
                  f"({100. * batch_idx / len(loaders['train']):.0f}%)]\tLoss: {loss.item():.6f}")


In [44]:

def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loaders['test']:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(loaders['test'].dataset)
    accuracy = 100. * correct / len(loaders['test'].dataset)
    print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(loaders['test'].dataset)} "
          f"({accuracy:.0f}%)")

In [45]:
for epoch in range(1, 11):
    train(epoch)
    test()

Train Epoch: 1 [0/42000 (0%)]	Loss: 2.327636
Train Epoch: 1 [2000/42000 (5%)]	Loss: 0.957982
Train Epoch: 1 [4000/42000 (10%)]	Loss: 0.736885
Train Epoch: 1 [6000/42000 (14%)]	Loss: 0.502011
Train Epoch: 1 [8000/42000 (19%)]	Loss: 0.313959
Train Epoch: 1 [10000/42000 (24%)]	Loss: 0.432818
Train Epoch: 1 [12000/42000 (29%)]	Loss: 0.249899
Train Epoch: 1 [14000/42000 (33%)]	Loss: 0.482792
Train Epoch: 1 [16000/42000 (38%)]	Loss: 0.331228
Train Epoch: 1 [18000/42000 (43%)]	Loss: 0.319606
Train Epoch: 1 [20000/42000 (48%)]	Loss: 0.167261
Train Epoch: 1 [22000/42000 (52%)]	Loss: 0.182800
Train Epoch: 1 [24000/42000 (57%)]	Loss: 0.134341
Train Epoch: 1 [26000/42000 (62%)]	Loss: 0.292086
Train Epoch: 1 [28000/42000 (67%)]	Loss: 0.138815
Train Epoch: 1 [30000/42000 (71%)]	Loss: 0.074641
Train Epoch: 1 [32000/42000 (76%)]	Loss: 0.201915
Train Epoch: 1 [34000/42000 (81%)]	Loss: 0.183508
Train Epoch: 1 [36000/42000 (86%)]	Loss: 0.218426
Train Epoch: 1 [38000/42000 (90%)]	Loss: 0.163059
Train Epoc

In [46]:
def generate_submission(model, test_loader, output_file="submission.csv"):
    """
    Generate predictions for the test set and save them in a CSV file for Kaggle submission.
    Parameters:
        model (torch.nn.Module): Trained PyTorch model.
        test_loader (DataLoader): DataLoader for the test dataset.
        output_file (str): Name of the output CSV file.
    """
    model.eval()  # Set the model to evaluation mode
    predictions = []
    
    with torch.no_grad():
        for data, _ in test_loader:  # DataLoader returns (data, _), but labels are not needed
            data = data.to(device)
            output = model(data)
            preds = output.argmax(dim=1, keepdim=False)  # Get the predicted class
            predictions.extend(preds.cpu().numpy())  # Move predictions to CPU and store them

    # Create the submission DataFrame
    submission = pd.DataFrame({
        "ImageId": range(1, len(predictions) + 1),  # Image IDs start at 1
        "Label": predictions  # Predicted labels
    })

    submission.to_csv(output_file, index=False)  # Save submission file
    print(f"Submission file saved as {output_file}")


In [47]:
generate_submission(model, loaders['test'], "submission.csv")


Submission file saved as submission.csv
